In [1]:
import nshutils as nu

nu.pretty()

In [14]:
import datasets

dataset = datasets.load_dataset("nimashoghi/mptrj", split="train")
dataset.set_format("numpy")
dataset

Dataset({
    features: ['numbers', 'positions', 'forces', 'cell', 'pbc', 'energy', 'stress', 'e_per_atom_relaxed', 'mp_id', 'energy_per_atom', 'ef_per_atom_relaxed', 'corrected_total_energy', 'ef_per_atom', 'task_id', 'calc_id', 'ionic_step', 'filename', 'extxyz_id', 'num_atoms', 'corrected_total_energy_relaxed', 'energy_referenced', 'corrected_total_energy_referenced', 'corrected_total_energy_relaxed_referenced', 'composition'],
    num_rows: 1559916
})

In [15]:
dataset["composition"][0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 5, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [16]:
import torch

nu.display(torch.bincount(torch.from_numpy(dataset["numbers"][0]), minlength=120))
nu.display(dataset["composition"][0])

In [17]:
from collections.abc import Callable

import numpy as np
from sklearn.linear_model import LinearRegression, Ridge


def get_species_ref_energy_by_linear_comb(
    compositions: np.ndarray,
    energies: np.ndarray,
    model_cls: Callable[[], Ridge | LinearRegression],
):
    """
    Total energy as y, composition as c_i,
    solve linear regression of y = c_i*X
    sklearn LinearRegression as solver

    x should be one-hot-indexed
    give num_chem_species if possible
    """

    c = compositions
    y = energies
    num_chem_species = c.shape[1]

    # tweak to fine tune training from many-element to small element
    zero_indices = np.all(c == 0, axis=0)
    c_reduced = c[:, ~zero_indices]
    full_coeff = np.zeros(num_chem_species)
    coef_reduced = model_cls().fit(c_reduced, y).coef_
    full_coeff[~zero_indices] = coef_reduced

    return full_coeff


ref_linear = get_species_ref_energy_by_linear_comb(
    dataset["composition"],
    dataset["energy"],
    lambda: LinearRegression(fit_intercept=False),
)
nu.display(ref_linear)

ref_ridge = get_species_ref_energy_by_linear_comb(
    dataset["composition"],
    dataset["energy"],
    lambda: Ridge(fit_intercept=False, alpha=0.1),
)
nu.display(ref_ridge)

In [18]:
nu.display(
    np.vstack(
        [
            ref_linear,
            ref_ridge,
            np.load("mptrj_linref.npy"),
            np.abs(ref_linear - ref_ridge),
            np.abs(ref_linear - np.load("mptrj_linref.npy")),
        ]
    )
)

In [6]:
np.load("mptrj_linref.npy")

array([-1.68470997e-17, -3.66698205e+00, -1.32293365e+00, -3.48009588e+00,
       -4.74774452e+00, -7.72465532e+00, -8.40652608e+00, -7.36064361e+00,
       -7.28499821e+00, -4.89743239e+00, -3.00579744e-02, -2.75732065e+00,
       -2.81323747e+00, -4.84514495e+00, -7.69467596e+00, -6.96312503e+00,
       -4.67204117e+00, -2.81088578e+00, -6.28330754e-02, -2.61566441e+00,
       -5.38863054e+00, -7.87437324e+00, -1.02689265e+01, -8.66271043e+00,
       -9.23543085e+00, -8.30401976e+00, -7.04723781e+00, -5.57891191e+00,
       -5.17152482e+00, -3.25211424e+00, -1.29147935e+00, -3.52818212e+00,
       -4.70975380e+00, -3.97641623e+00, -3.88538728e+00, -2.51773914e+00,
        6.76992808e+00, -2.56441071e+00, -4.93910306e+00, -1.01526380e+01,
       -1.18499238e+01, -1.21401382e+01, -8.78984237e+00, -8.78738397e+00,
       -7.77604301e+00, -6.85030040e+00, -4.89354004e+00, -2.06389151e+00,
       -6.41175260e-01, -2.78881612e+00, -3.81828349e+00, -3.58734526e+00,
       -2.87770288e+00, -1.63306952e+00,  9.81037214e+00, -2.76731275e+00,
       -4.98707742e+00, -8.93569238e+00, -8.73706832e+00, -8.01640053e+00,
       -8.25298639e+00, -7.59756826e+00, -8.17025732e+00, -1.35933187e+01,
       -1.85186549e+01, -7.64332721e+00, -8.13251516e+00, -7.61683309e+00,
       -6.83831699e+00, -7.82245038e+00, -3.57834799e+00, -7.44738275e+00,
       -1.27930610e+01, -1.41114391e+01, -9.36038961e+00, -1.13909903e+01,
       -9.61262732e+00, -7.31823610e+00, -5.30230792e+00, -2.37749511e+00,
        2.49841340e-01, -2.32495444e+00, -3.73430081e+00, -3.43997937e+00,
        0.00000000e+00,  1.77635684e-15,  0.00000000e+00,  0.00000000e+00,
        6.24500451e-16, -5.07883418e+00, -1.10214086e+01, -1.22493988e+01,
       -1.38558148e+01, -1.49205759e+01, -1.52700887e+01,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00])